Question 1

In [412]:
import numpy as np

In [413]:
def generateData(d, N, noise=0.1):
    np.random.seed(12345)
    X = np.concatenate([np.ones([N, 1]), np.random.randn(N, d)], axis=1)
    lambdaTrue = -1 + 2 * np.tan(np.random.randn(len(X[0])))
    Y = 1 * ((X @ lambdaTrue + noise * np.random.randn(N)) > 0)
    D = X, Y
    return D

In [414]:
def split(D, train_size):
    x, y = D
    x_trainning = []
    x_testing = []
    y_traning = []
    y_testing = []
    for i in range(0, train_size):
        x_trainning.append(x[i])
        y_traning.append(y[i])
    for i in range(train_size, len(y)):
        x_testing.append(x[i])
        y_testing.append(y[i])
    return x_trainning, x_testing, y_traning, y_testing

In [415]:
def model(x, par):
    z = x @ par
    # return 1 / (np.exp(-z)+1)
    return np.power(1 / (1 + np.power(np.e, -z)), -1)

In [416]:
def dModel(x, par):
    f = np.power((1 + np.power(np.e, (np.dot(x, par)))), -1)
    gradient = np.dot(f * (1 - f), x)
    return gradient

In [417]:
def objective(par, data):
    ell = 0
    X, Y = data
    for n in range(0, len(X)):
        x, y = X[n], Y[n]
        # f = np.power((np.dot(par,x)),np.e)
        f = model(X, par)
        if y == 1:
            f = (f)
        else:
            deltat = (1 - f)
        s = -(np.log(f))
        ell = ell + s
    return ell

In [418]:
def gradient(par, data):
    grad = np.zeros(len(par))
    X, Y = data
    for n in range(0, len(X)):
        f = model(X[n], par)
        x, y = X[n], Y[n]
        # deltaf = (dModel(X,par))
        s = -(y - f) * (dModel(x, par)) / (y * f + (1 - y) * (1 - f))
        # s = -(y-f)/(y*f+(1-y)*(1-f))
        grad = grad + s
    return grad

In [419]:
def train(par0, eta, T, data):
    par = par0
    obj = []
    for t in range(0, T):
        ell = objective(par, data)
        obj.append(ell)
        grad = gradient(par, data)
        par = par - grad * eta
    return par, obj

In [420]:
lambda1 = np.transpose(np.full(11, 0.01))
d = 10
N = 100
D = generateData(d, N)
x_trainning, x_testing, y_traning, y_testing = split(D, 50)
eta = 0.01
epoch = []
itterations = 5
# tryInit = [np.random.randn(len(X[0]))for m in range(5)]
for i in range(0, itterations):
    par, values = train(np.random.normal(0, 1, len(x_trainning[0])), eta, itterations, (x_trainning, y_traning))
    epoch.append((par, values))

In [421]:
def ER(par, data):
    ER = 0
    X, Y = data
    for n in range(0, len(D) - 1):
        x, y = X[n], Y[n]
        yHat = model(x, par[n])
        s = 1 * (y != yHat)
        ER = ER + s
    return ER / len(data)

In [422]:
ERs = []
for i in epoch:
    ERs.append(ER(i, (x_testing, y_testing)))
print(ERs)

[0.5, 0.5, 0.5, 0.5, 0.5]


Question 2

In [423]:
import pandas as pd

In [424]:
data = pd.read_csv("CarsDataSetForCW1.csv",sep =",")
Z = data[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].to_numpy()
Y = 1 * (Z[:, 0] >= np.mean(Z[:, 0]))
X, Y = Z[:, 1 :], Y
print(Z)

[[ 18.    8.  307.  ...  12.   70.    1. ]
 [ 15.    8.  350.  ...  11.5  70.    1. ]
 [ 18.    8.  318.  ...  11.   70.    1. ]
 ...
 [ 32.    4.  135.  ...  11.6  82.    1. ]
 [ 28.    4.  120.  ...  18.6  82.    1. ]
 [ 31.    4.  119.  ...  19.4  82.    1. ]]


In [425]:
D_train = (X[:len(data)//2], Y[:len(data)//2])
D_test = (X[len(data)//2:], Y[len(data)//2:])

In [427]:
D_train_normalised = ((D_train[0] - np.average(D_train[0])) / np.std(D_train[0]), D_train[1])
print(D_train_normalised)
D_test_normalised = ((D_test[0] - np.average(D_test[0])) / np.std(D_test[0]), D_test[1])
print(D_test_normalised)
lambda1 = np.transpose(np.full(198, 0.01))
d = 10
N = 100
D = generateData(d, N)


x_testing=D_test_normalised[0]
y_testing=D_test_normalised[1]
x_trainning=D_train_normalised[0]
y_traning=D_train_normalised[1]

eta = 0.01
epoch = []
itterations = 5
# tryInit = [np.random.randn(len(X[0]))for m in range(5)]
for i in range(0, itterations):
    par, values = train(np.random.normal(0, 1, len(x_trainning)), eta, itterations, (x_trainning, y_traning))
    epoch.append((par, values))
ERs = []
for i in epoch:
    ERs.append(ER(i, (x_testing, y_testing)))
print(ERs)

(array([[-0.44347148, -0.18249136, -0.33698461, ..., -0.43998011,
        -0.3893552 , -0.44958138],
       [-0.44347148, -0.1449591 , -0.3064351 , ..., -0.44041653,
        -0.3893552 , -0.44958138],
       [-0.44347148, -0.17289009, -0.31952775, ..., -0.44085295,
        -0.3893552 , -0.44958138],
       ...,
       [-0.44696285, -0.37626255, -0.40506638, ..., -0.43107711,
        -0.38411814, -0.44958138],
       [-0.44696285, -0.36491559, -0.39808363, ..., -0.43116439,
        -0.38411814, -0.44958138],
       [-0.44696285, -0.37189834, -0.3893552 , ..., -0.43805985,
        -0.38411814, -0.44870854]]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 198 is different from 7)